In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu

import plotly.express as px

import Models.Soybean_USA_Yield_GA as sy

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope)
raw_data.keys()

dict_keys(['years', 'locations', 'yield', 'weights', 'planting_progress', 'blooming_progress', 'w_df_all', 'w_w_df_all'])

In [8]:
w_w_df_all=raw_data['w_w_df_all']['hist']

dt_s=dt(2022,5,1)
dt_e=dt(2022,5,6)

freq_start='1D'
freq_end='1D'

ref_year_s=dt(2022,11,4)

multi_ww_df=um.generate_weather_windows_df(w_w_df_all, date_start=dt_s, date_end=dt_e, ref_year_start=ref_year_s, freq_start=freq_start, freq_end=freq_end)

100%|██████████| 6/6 [00:00<00:00, 154.14it/s]


In [10]:
multi_ww_df.head(2)

,USA_TempMax_May01-May01,USA_Sdd30_May01-May01,USA_Prec_May01-May01,USA_TempMax_May01-May02,USA_Sdd30_May01-May02,USA_Prec_May01-May02,USA_TempMax_May01-May03,USA_Sdd30_May01-May03,USA_Prec_May01-May03,USA_TempMax_May01-May04,...,USA_Prec_May04-May06,USA_TempMax_May05-May05,USA_Sdd30_May05-May05,USA_Prec_May05-May05,USA_TempMax_May05-May06,USA_Sdd30_May05-May06,USA_Prec_May05-May06,USA_TempMax_May06-May06,USA_Sdd30_May06-May06,USA_Prec_May06-May06
year,,,,,,,,,,,,,,,,,,,,,
1985,23.115531,0.0,7.740059,23.115531,0.0,7.740059,23.115531,0.0,7.740059,23.115531,...,0.172154,24.673521,0.0,0.451683,24.673521,0.0,0.451683,23.908775,0.0,3.123178
1986,21.501045,0.0,8.649489,21.501045,0.0,8.649489,21.501045,0.0,8.649489,21.501045,...,0.171409,25.347209,0.0,0.152654,25.347209,0.0,0.152654,26.536711,0.0,0.870573
